**Importing and Reading**

In [74]:
import pandas as pd
import numpy as np

df = pd.read_csv('df_activity.csv', index_col = 0)
df.head()

,e1,e2,e3,e4,e5,e6,a1,a2,a3,w1,w2,w3,Label
0,-0.231112,-0.170864,0.250235,0.269120,0.494601,0.107579,0.112402,0.275193,0.280864,-1.549786,-1.434876,-0.693274,Null
1,-0.229855,-0.174416,0.251058,0.270839,0.485838,0.107579,0.112957,0.277866,0.282306,-1.585254,-1.478693,-0.706290,Null
2,-0.228599,-0.177968,0.251881,0.272558,0.477076,0.107579,0.112129,0.276654,0.277062,-1.648361,-1.417204,-0.721149,Null
3,-0.227342,-0.181521,0.252704,0.274277,0.468313,0.107579,0.112407,0.277985,0.277296,-1.746202,-1.326161,-0.677304,Null
4,-0.226085,-0.185073,0.253527,0.275996,0.459551,0.107579,0.112685,0.279317,0.277530,-1.864187,-1.165475,-0.594791,Null


**Encoding the Label**

In [75]:
from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()
df['Label'] = label.fit_transform(df['Label'].values)

#To check corresponding label values
#df['Label_encoded'] = LabelEncoder().fit_transform(df['Label'].values)
#df

**Sliding Window**

In [76]:
def sliding_window_samples(data, samples_per_window, overlap_ratio):
    windows = []
    indices = []
    curr =  0
    win_len = samples_per_window
    if overlap_ratio !=  None :
        overlapping_elements = int((overlap_ratio / 100) * (win_len))
    if overlapping_elements >= win_len:
        print('Number of overlapping elements exceeds window size.')
    while (curr < len(data) - win_len):
         windows.append(data[curr:curr + win_len])
         indices.append([curr, curr + win_len])
         curr = curr + win_len - overlapping_elements
         
         try:
            result_windows = np.array(windows)
            result_indices = np.array(indices)
         except:
             result_windows = np.empty( shape =(len(windows), win_len, data.shape[1]), dtype =object)
             result_indices = np.array(indices)
             for i in range(0, len(windows)):
                result_windows[i] = windows[i]
                result_indices[i] = indices[i]
    return result_windows, result_indices

In [77]:
window_length = 100
window_skip = 50
df_1, df_2 = sliding_window_samples(df, window_length, window_skip)
data = np.array(df_1, dtype = float)
data.shape

(1163, 300, 13)

**Extracting features and target**

In [78]:
from sklearn.model_selection import train_test_split

X = data[:, : ,0:12]
y = data[:, : , 12]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123, shuffle = False)
X_train.shape,  X_test.shape, y_train.shape, y_test.shape

((930, 300, 12), (233, 300, 12), (930, 300), (233, 300))

In [79]:
def dim_reduction(s):
    axis = 1
    y, indices = np.unique(s, return_inverse=True)
    y_new = y[np.argmax(np.apply_along_axis(np.bincount, axis, indices.reshape(s.shape),None, np.max(indices) + 1), axis=axis)]
    return y_new

In [80]:
#X1_train = np.reshape(X_train, (X_train.shape[0], -1))
#X1_test = np.reshape(X_test, (X_test.shape[0], -1))

y1_train = dim_reduction(y_train)
y1_test = dim_reduction(y_test)
#X1_train.shape

**Model**

In [81]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [87]:
model = Sequential()
model.add(LSTM(256, input_shape = X_train[0].shape , activation = 'relu', return_sequences = True))
model.add(Dropout(0.1))
model.add(LSTM(128, activation = 'relu', return_sequences = True))
model.add(Dropout(0.1))
model.add(LSTM(64,activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(6, activation = 'softmax'))

In [83]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [84]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_28 (LSTM)               (None, 300, 256)          275456    
_________________________________________________________________
dropout_27 (Dropout)         (None, 300, 256)          0         
_________________________________________________________________
lstm_29 (LSTM)               (None, 300, 128)          197120    
_________________________________________________________________
dropout_28 (Dropout)         (None, 300, 128)          0         
_________________________________________________________________
lstm_30 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dense_27 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_29 (Dropout)         (None, 64)              

In [85]:
model.fit(X_train, y1_train, verbose = 1, validation_data = 0.2)

ValueError: in user code:

    C:\anaconda\lib\site-packages\tensorflow\python\keras\engine\training.py:855 train_function  *
        return step_function(self, iterator)
    C:\anaconda\lib\site-packages\tensorflow\python\keras\engine\training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\anaconda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\anaconda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\anaconda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\anaconda\lib\site-packages\tensorflow\python\keras\engine\training.py:838 run_step  **
        outputs = model.train_step(data)
    C:\anaconda\lib\site-packages\tens